In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Data Cleaning

In [30]:
df = pd.read_csv('MTA_Daily_Ridership_Data.csv')
df.head()

,Date,Subways: Total Estimated Ridership,Subways: % of Comparable Pre-Pandemic Day,Buses: Total Estimated Ridership,Buses: % of Comparable Pre-Pandemic Day,LIRR: Total Estimated Ridership,LIRR: % of Comparable Pre-Pandemic Day,Metro-North: Total Estimated Ridership,Metro-North: % of Comparable Pre-Pandemic Day,Access-A-Ride: Total Scheduled Trips,Access-A-Ride: % of Comparable Pre-Pandemic Day,Bridges and Tunnels: Total Traffic,Bridges and Tunnels: % of Comparable Pre-Pandemic Day,Staten Island Railway: Total Estimated Ridership,Staten Island Railway: % of Comparable Pre-Pandemic Day
0,03/01/2020,2212965,0.97,984908,0.99,86790,1.00,55825,0.59,19922,1.13,786960,0.98,1636,0.52
1,03/02/2020,5329915,0.96,2209066,0.99,321569,1.03,180701,0.66,30338,1.02,874619,0.95,17140,1.07
2,03/03/2020,5481103,0.98,2228608,0.99,319727,1.02,190648,0.69,32767,1.10,882175,0.96,17453,1.09
3,03/04/2020,5498809,0.99,2177165,0.97,311662,0.99,192689,0.70,34297,1.15,905558,0.98,17136,1.07
4,03/05/2020,5496453,0.99,2244515,1.00,307597,0.98,194386,0.70,33209,1.12,929298,1.01,17203,1.08


In [31]:
columns = [
    'Subways: Total Estimated Ridership',
    'Buses: Total Estimated Ridership',
    'LIRR: Total Estimated Ridership',
    'Metro-North: Total Estimated Ridership',
    'Staten Island Railway: Total Estimated Ridership',
]

In [32]:
# Filter DataFrame in place to keep only the specified columns plus Date
df = df[['Date'] + columns]
df.head()

,Date,Subways: Total Estimated Ridership,Buses: Total Estimated Ridership,LIRR: Total Estimated Ridership,Metro-North: Total Estimated Ridership,Staten Island Railway: Total Estimated Ridership
0,03/01/2020,2212965,984908,86790,55825,1636
1,03/02/2020,5329915,2209066,321569,180701,17140
2,03/03/2020,5481103,2228608,319727,190648,17453
3,03/04/2020,5498809,2177165,311662,192689,17136
4,03/05/2020,5496453,2244515,307597,194386,17203


In [33]:
df['Date'] = pd.to_datetime(df['Date'], format="%m/%d/%Y")

**Adding Temporal Features**

In [34]:
df['Day of Week'] = df['Date'].dt.dayofweek  # 0=Monday, 6=Sunday
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df['Is Weekend'] = df['Day of Week'].isin([5,6]).astype(int)
df.head()

,Date,Subways: Total Estimated Ridership,Buses: Total Estimated Ridership,LIRR: Total Estimated Ridership,Metro-North: Total Estimated Ridership,Staten Island Railway: Total Estimated Ridership,Day of Week,Month,Year,Is Weekend
0,2020-03-01,2212965,984908,86790,55825,1636,6,3,2020,1
1,2020-03-02,5329915,2209066,321569,180701,17140,0,3,2020,0
2,2020-03-03,5481103,2228608,319727,190648,17453,1,3,2020,0
3,2020-03-04,5498809,2177165,311662,192689,17136,2,3,2020,0
4,2020-03-05,5496453,2244515,307597,194386,17203,3,3,2020,0


In [52]:
import holidays

us_holidays = holidays.US()
# Convert datetime index to date objects for proper comparison with holidays
df['Is Holiday'] = [1 if date in us_holidays else 0 for date in df['Date'].dt.date]

**Add External Event Features**: COVID, Hurricanes/Floods, Concerts and Sports events

In [54]:
covid_start = pd.to_datetime('2020-03-15')
covid_end = pd.to_datetime('2020-06-30')
df['COVID'] = ((df['Date'] >= covid_start) & (df['Date'] <= covid_end)).astype(int)


In [66]:
NY_HURRICANE_DATES = pd.Series([
    '2020-07-10',
    '2020-07-11', # Tropical Storm Fay 
    '2020-08-04', # Hurricane Isaias
    '2020-08-31', # Hurricane Laura
    '2020-09-21', # Hurricane Teddy
    '2020-09-22',
    '2021-07-09', # Hurricane Elsa
    '2021-08-18', # Tropical Storm Fred
    '2021-08-22', # Hurricane Henri
    '2021-09-01', # Hurricane Ida
    '2022-10-01', # Hurricane Nicole
    '2022-10-02', # Hurricane Nicole
    '2022-10-03', # Hurricane Nicole
    '2022-10-04', # Hurricane Nicole
    '2023-09-28', # Tropical Storm Ophelia
    '2023-09-29',
    '2020-09-30',
    '2024-08-07', # Hurricane Beryl
    '2024-08-08', # Hurricane Beryl
    '2024-08-09', # Hurricane Beryl
    '2025-07-07', # Tropical Storm Chantal
    '2025-07-08', # Tropical Storm Chantal
])

In [69]:
NY_HURRICANE_DATES = pd.to_datetime(NY_HURRICANE_DATES, format="%Y-%m-%d")
# Use isin method for proper pandas comparison
df['Hurricane'] = df['Date'].isin(NY_HURRICANE_DATES).astype(int)

In [71]:
# US Open Tennis Tournament date ranges
us_open_ranges = [
    ('2024-08-19', '2024-09-09'),
    ('2023-08-22', '2023-09-10'), 
    ('2022-08-23', '2022-09-12'),
    ('2021-08-24', '2021-09-13'),
    ('2020-08-31', '2020-09-13')
]

# Generate all dates within the US Open ranges
us_open_dates = []
for start_date, end_date in us_open_ranges:
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    us_open_dates.extend(date_range)

US_OPEN_DATES = pd.Series(us_open_dates)
df['US_Open'] = df['Date'].isin(US_OPEN_DATES).astype(int)

**Create Lag Features** Tree models cannot handle sequences directly, so we create lagged values and rolling averages for each mode.